In [1]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("../data/raw/itineraires_alpes.csv")
print(f"Chargés : {len(df)} itinéraires")
df.head()

Chargés : 161 itinéraires


,name,massif,lat,lon,denivele_positif,exposition,difficulty_ski
0,Aiguille du Midi,Chamonix,45.9234,6.8683,1200,N,S3
1,Col des Flambeaux,Chamonix,45.9170,6.8900,800,E,S2
2,Vallée Blanche,Chamonix,45.9100,6.8800,600,S,S1
3,Argentière Glacier,Chamonix,45.9500,6.9500,1500,N,S4
4,Pointe Helbronner,Chamonix,45.8300,6.9000,1000,SE,S3


In [3]:
BULLETINS_AVY = {
    "Chamonix": 3,
    "Vanoise": 2,
    "Écrins": 3,
    "Suisse": 3,
    "Italie": 2
}

def get_avy_risk(massif):
    return BULLETINS_AVY.get(massif, 3) / 5.0

In [4]:
# 3. Fonction vent OpenWeather (exemple simplifié – on prend un point central par massif)
WIND_CACHE = {}  # Pour éviter 150 appels API
def get_wind_score(lat, lon):
    key = f"{lat},{lon}"
    if key in WIND_CACHE:
        return WIND_CACHE[key]
    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={open('../.env').read().strip().split('=')[1]}&units=metric"
        data = requests.get(url, timeout=5).json()
        wind = data["wind"]["speed"]  # m/s
        gust = data.get("wind", {}).get("gust", wind)
        wind_score = min(gust / 20.0, 1.0)  # >20 m/s = max pénalité
        WIND_CACHE[key] = wind_score
        return wind_score
    except:
        return 0.5  # pénalité moyenne si erreur

# 4. Fonction scoring finale v2 (la vraie, celle qu’on veut)
def scoring_v2(row, niveau="S3", dplus_max=1400):
    # --- DANGER (0 = safe, 1 = très dangereux) ---
    # 1. Risque avy live
    avy_risk = get_avy_risk(row["massif"])
    
    # 2. Vent live
    wind_score = get_wind_score(row["lat"], row["lon"])
    
    # 3. Exposition (faces sud = pire regel)

In [ ]:
print("Calcul des scores (patience 15-20 sec la première fois)...")
results = df.apply(scoring_v2, axis=1, args=("S3", 1400))
df["final_score"], df["danger"], df["avy_risk"], df["wind_kmh"] = zip(*results)

Calcul des scores (patience 15-20 sec la première fois)...


In [ ]:
top10 = df.sort_values("final_score", ascending=False).head(10)
display(top10[["name", "massif", "denivele_positif", "exposition", 
               "difficulty_ski", "avy_risk", "wind_kmh", "danger", "final_score"]].round(2))